# EWX API Test

                                ''' EWX API test '''
                '''notebook to show how to make requests to EWX api'''
    '''inputs: bearer token (from EWX console/API docs/try out/execute/response)'''
    '''ouputs: curl wrapper to make API req to EWX, given bearer token (and url)'''
                           '''returns .json of results'''

## import modules

In [50]:
import os
import time
import math
import json
import pickle
import requests
import pandas as pd
import numpy as np
import pprint
import datetime as dt

from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
pp = pprint.PrettyPrinter(indent = 1)
requests.packages.urllib3.disable_warnings()

base = os.getcwd()

# requests Module

need bearer token from running API command in EWX console (expires after 100s)

In [51]:
bearer = 'ya29.GlxQB825JWOQYNTECogq9pH0vJMtyQybxZ7_zwOTr_-PybJ7lM5P5gPHJhYfL6_6tR9x6Mbc5uVqdEoSTWwXYFXhtihc0Z6fCoPWao57j0olF9Rjhj-fvoMU-jHahA'
bearer_auth = ''.join(['Bearer ', bearer])

print('authenticating with {}.'.format(bearer_auth))

authenticating with Bearer ya29.GlxQB825JWOQYNTECogq9pH0vJMtyQybxZ7_zwOTr_-PybJ7lM5P5gPHJhYfL6_6tR9x6Mbc5uVqdEoSTWwXYFXhtihc0Z6fCoPWao57j0olF9Rjhj-fvoMU-jHahA.


## Make curl request to send API

## header of get request

    what url does the request GET from? hint: look at EWX API docs...

    {'Authorization': 'Bearer '
                          'ya29.GlxQB825JWOQYNTECogq9pH0vJMtyQybxZ7_zwOTr_-PybJ7lM5P5gPHJhYfL6_6tR9x6Mbc5uVqdEoSTWwXYFXhtihc0Z6fCoPWao57j0olF9Rjhj-fvoMU-jHahA',
     'GET': 'https://api.cloud.energyworx.com/_ah/api/ewx/v1/user/get',
     'X-NAMESPACE': 'na.engie.com',
     'accept': 'application/json'}

In [53]:
auth = ("Authorization", bearer_auth)
ns = ("X-NAMESPACE", "na.engie.com")
api_req = ("GET", "https://api.cloud.energyworx.com/_ah/api/ewx/v1/user/get")
resp = ("accept", "application/json")

## Get refresh token for user

In [55]:
def get_user_token(bearer):
    header = {'Authorization':bearer, 'X-NAMESPACE':'na.engie.com', 'accept':'application/json'}

    user = 'https://api.cloud.energyworx.com/_ah/api/ewx/v1/user/get'

    user_info = requests.get(user, headers = header, verify = False).json()
    refresh_token = user_info['refreshToken']
    return(refresh_token)

In [56]:
refresh = get_user_token(bearer_auth)
print('refresh token is {}'.format(refresh))

refresh token is 1/7xPGFuXNRaRjWCFyYHRHdzrdaJuRXHB4R4TRvdnjPEg


## update_refresh()

need to update bearer token after expire given refresh, this gets EWX refresh

In [45]:
#update = 'api.cloud.energyworx.com/_ah/api'
#h = {'grant_type': 'refresh_token', }
#print('user refresh token {}.'.format(user_info['refreshToken']))
#return(refresh_token)

## get_release_hist()

In [36]:
def get_release_hist(bearer_auth):
    header = {'Authorization':bearer_auth, 'X-NAMESPACE':'na.engie.com', 'accept':'application/json'}

    user = 'https://api.cloud.energyworx.com/_ah/api/ewx/v1/system/version/release/history'

    payload = requests.get(user, headers = header, verify = False).json()

    try:
        pgtoken = payload['pageToken']
    except:
        pgtoken = None
        
    releases = payload['releases']
    
    release_df = pd.DataFrame(releases)
    return(pgtoken, release_df)

In [37]:
pg, release = get_release_hist(bearer_auth)

In [38]:
release.head()

,id,items,releasedDatetime,version
0,v20190724-r0,"[{'item': 'feature', 'summary': 'Merged in rel...",2019-07-24T23:10:50.000000,v20190724-r0
1,v20190719-r6,"[{'item': 'fix', 'summary': 'Merged in hotfix/...",2019-07-24T18:01:03.000000,v20190719-r6
2,v20190719-r5,"[{'item': 'fix', 'summary': 'Merged in hotfix/...",2019-07-23T23:17:13.000000,v20190719-r5
3,v20190719-r4,"[{'item': 'fix', 'summary': 'Merged in hotfix/...",2019-07-22T19:39:37.000000,v20190719-r4
4,v20190719-r3,"[{'item': 'fix', 'summary': 'Merged in hotfix/...",2019-07-22T14:22:21.000000,v20190719-r3


In [39]:
release.tail()

,id,items,releasedDatetime,version
15,v20190626-r1,"[{'item': 'fix', 'summary': 'Merged in hotfix/...",2019-06-27T20:34:33.000000,v20190626-r1
16,v20190626-r0,"[{'item': 'feature', 'summary': 'Merged in rel...",2019-06-27T09:33:57.000000,v20190626-r0
17,v20190624-r0,"[{'item': 'feature', 'summary': 'Merged in rel...",2019-06-25T03:50:37.000000,v20190624-r0
18,v20190623-r2,"[{'item': 'fix', 'summary': 'Merged in hotfix/...",2019-06-24T20:50:10.000000,v20190623-r2
19,v20190623-r1,"[{'item': 'fix', 'summary': 'Merged in hotfix/...",2019-06-24T17:32:18.000000,v20190623-r1


## Turn items to dataframe

In [40]:
items = []

for index, item in enumerate(release['items']):
    item_df = pd.DataFrame(item)
    items.append(item_df)

In [41]:
items[0]

,item,summary
0,feature,Merged in release/v20190724-r0 (pull request #...
1,feature,Merged in feature/one_to_n (pull request #1549)
2,feature,Merged in feature/setup_namespace (pull reques...
3,fix,Merged in feature/fix-bulk-issue (pull request...
4,fix,Merged in feature/telemetered-consumption-fixe...
5,feature,Merged in feature/market-adapter-excel (pull r...
6,feature,Merged in feature/transform-enhancement (pull ...
7,feature,Merged in feature/market-adapter-switching (pu...
8,fix,Merged in feature/market-adapter-fixed-width (...
9,fix,[FIXED] bulk meter payload with 1000 meters


## Pandas Excel writer

Use pandas to write dataframe to xlsx sheet.

### Sample

    df1 = pd.DataFrame([['a', 'b'], ['c', 'd']],
                       index=['row 1', 'row 2'],
                       columns=['col 1', 'col 2'])

    df1.to_excel("output.xlsx")  # doctest: +SKIP

    df1.to_excel("output.xlsx", sheet_name='Sheet_name_1')  # doctest: +SKIP

    df2 = df1.copy()
    with pd.ExcelWriter('output.xlsx') as writer:  # doctest: +SKIP
        df1.to_excel(writer, sheet_name='Sheet_name_1')
        df2.to_excel(writer, sheet_name='Sheet_name_2')

    #want main release df to be xlsx spreadsheet
    #with all items cells as their own sheet from each items df

In [43]:
release.head()
print('release df of shape: {}.'.format(release.shape))
print('items list len: {}.'.format(len(items)))

release df of shape: (20, 4).
items list len: 20.


In [49]:
with pd.ExcelWriter('ewx-release-hist.xlsx') as w:
    release.to_excel(w, sheet_name = 'release_hist')
    name = [str(a) for a in release.id]
    
    for index, item in enumerate(items):
        item.to_excel(w, sheet_name = str(name[index]))